Environment Set Up


In [1]:
!pip install jiwer

In [2]:
!pip install tensorboard

In [3]:
!pip install peft==0.7.1 accelerate==0.24.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 27.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.12.0
    Uninstalling accelerate-1.12.0:
      Successfully uninstalled accelerate-1.12.0
  Attempting uninstall: peft
    Found existing installation: peft 0.18.0
    Uninstalling peft-0.18.0:
      Successfully uninstalled peft-0.18.0


In [4]:
!pip install transformers==4.36.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 147.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 123.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


In [5]:
!pip install datasets torchaudio librosa soundfile jiwer gradio

In [6]:
!pip install torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.6 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load Common Voice 23.0

In [8]:


import os, sys, math, random, json, shutil, time
from pathlib import Path
import numpy as np, pandas as pd
import torch
print('torch', torch.__version__, 'cuda available:', torch.cuda.is_available())
DRIVE_ROOT = '/content/drive/MyDrive/ibibio_asr'
os.makedirs(DRIVE_ROOT, exist_ok=True)

LOCAL_TSV_DIR = os.path.join(DRIVE_ROOT, 'common_voice_tsvs')
COMMON_VOICE_DIR = os.path.join(DRIVE_ROOT, 'common_voice_23_0_ibb')
OUTPUT_DIR = os.path.join(DRIVE_ROOT, 'wav2vec2_xlsr_optionA')
os.makedirs(OUTPUT_DIR, exist_ok=True)

PRETRAINED_MODEL = 'facebook/wav2vec2-large-xlsr-53'
SAMPLE_RATE = 16000
MIN_AUDIO = 0.5
MAX_AUDIO = 30.0

# Pretraining vs finetuning params (defaults small for Colab)
PRETRAIN_EPOCHS = 1   # set low for Colab testing; raise for serious pretraining
FINETUNE_EPOCHS = 3
PRETRAIN_BATCH = 8
FINETUNE_BATCH = 4

print('Configuration set. OUTPUT_DIR=', OUTPUT_DIR)


torch 2.9.0+cu126 cuda available: True
Configuration set. OUTPUT_DIR= /content/drive/MyDrive/ibibio_asr/wav2vec2_xlsr_optionA


In [9]:
from datasets import load_dataset, DatasetDict, Dataset, Audio
import pandas as pd
import os
import librosa
import soundfile as sf

CV_PATH = '/content/drive/MyDrive/Ibibio_Voice/Data/ibb/'

def get_duration_fixed(example):
    """Universal duration function that works with Audio objects"""
    try:
        # Get the actual file path from various possible structures
        if hasattr(example['path'], 'path'):
            file_path = example['path'].path
        elif isinstance(example['path'], dict) and 'path' in example['path']:
            file_path = example['path']['path']
        else:
            file_path = example['path']

        if file_path and os.path.exists(file_path):
            duration = librosa.get_duration(filename=file_path)
            return float(duration)
        else:
            print(f"File not found: {file_path}")
            return 0.0
    except Exception as e:
        print(f"Error getting duration: {e}")
        return 0.0

def load_all_datasets_with_duration_filtering():
    """Load all datasets with duration filtering applied"""

    # 1. Load the main splits (dev, test, train)
    main_splits = {}
    for split_name in ['train', 'test', 'dev']:
        tsv_path = os.path.join(CV_PATH, f"{split_name}.tsv")
        if os.path.exists(tsv_path):
            df = pd.read_csv(tsv_path, sep='\t')
            sample = df["path"].iloc[0]
            # Fix paths
            if sample.startswith("clips/"):   # CV-style relative path
                df["path"] = df["path"].apply(lambda p: os.path.join(CV_PATH, p))
            elif sample.startswith("common_voice_"):  # file names only
                df["path"] = df["path"].apply(lambda p: os.path.join(CV_PATH, "clips", p))
            else:
                # Possible absolute path — leave unchanged
                df["path"] = df["path"]
            dataset = Dataset.from_pandas(df)

            # Add duration and filter
            dataset = dataset.map(lambda x: {'duration': get_duration_fixed(x)})
            dataset = dataset.filter(lambda x: MIN_AUDIO <= x['duration'] <= MAX_AUDIO)

            main_splits[split_name] = dataset
            print(f"Loaded {split_name}: {len(dataset)} samples (after duration filtering)")

    # 2. Load validated and invalidated TSVs with duration filtering
    additional_datasets = {}

    # Validated data (high quality)
    validated_tsv = os.path.join(CV_PATH, "validated.tsv")
    if os.path.exists(validated_tsv):
        validated_df = pd.read_csv(validated_tsv, sep='\t')
        sample = validated_df["path"].iloc[0]
        # Fix paths
        if sample.startswith("clips/"):   # CV-style relative path
            validated_df["path"] = validated_df["path"].apply(lambda p: os.path.join(CV_PATH, p))
        elif sample.startswith("common_voice_"):  # file names only
            validated_df["path"] = validated_df["path"].apply(lambda p: os.path.join(CV_PATH, "clips", p))
        else:
            # Possible absolute path — leave unchanged
            validated_df["path"] = validated_df["path"]
        dataset = Dataset.from_pandas(validated_df)

        # Add duration and filter
        dataset = dataset.map(lambda x: {'duration': get_duration_fixed(x)})
        dataset = dataset.filter(lambda x: MIN_AUDIO <= x['duration'] <= MAX_AUDIO)

        additional_datasets['validated'] = dataset
        print(f"Loaded validated: {len(dataset)} samples (after duration filtering)")

    # Invalidated data (lower quality but still useful for pretraining)
    invalidated_tsv = os.path.join(CV_PATH, "invalidated.tsv")
    if os.path.exists(invalidated_tsv):
        invalidated_df = pd.read_csv(invalidated_tsv, sep='\t')
        sample = invalidated_df["path"].iloc[0]
        # Fix paths
        if sample.startswith("clips/"):   # CV-style relative path
            invalidated_df["path"] = invalidated_df["path"].apply(lambda p: os.path.join(CV_PATH, p))
        elif sample.startswith("common_voice_"):  # file names only
            invalidated_df["path"] = invalidated_df["path"].apply(lambda p: os.path.join(CV_PATH, "clips", p))
        else:
            # Possible absolute path — leave unchanged
            invalidated_df["path"] = invalidated_df["path"]
        dataset = Dataset.from_pandas(invalidated_df)

        # Add duration and filter
        dataset = dataset.map(lambda x: {'duration': get_duration_fixed(x)})
        dataset = dataset.filter(lambda x: MIN_AUDIO <= x['duration'] <= MAX_AUDIO)

        additional_datasets['invalidated'] = dataset
        print(f"Loaded invalidated: {len(dataset)} samples (after duration filtering)")

    return main_splits, additional_datasets

In [10]:
# Load all data with duration filtering
print("=== Loading datasets with duration filtering ===")
main_splits, additional_datasets = load_all_datasets_with_duration_filtering()
main_splits

=== Loading datasets with duration filtering ===


Map:   0%|          | 0/352 [00:00<?, ? examples/s]

/tmp/ipython-input-1453531509.py:21: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


Filter:   0%|          | 0/352 [00:00<?, ? examples/s]

Loaded train: 352 samples (after duration filtering)


...

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

/tmp/ipython-input-1453531509.py:21: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=file_path)


Filter:   0%|          | 0/115 [00:00<?, ? examples/s]

Loaded invalidated: 23 samples (after duration filtering)


{'train': Dataset({
     features: ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment', 'duration'],
     num_rows: 352
 }),
 'test': Dataset({
     features: ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment', 'duration'],
     num_rows: 328
 }),
 'dev': Dataset({
     features: ['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment', 'duration'],
     num_rows: 316
 })}

In [11]:
train_raw = main_splits.get("train", Dataset.from_dict({}))
test_raw = main_splits.get("test", Dataset.from_dict({}))
dev_raw  = main_splits.get("dev",  Dataset.from_dict({}))

validated_raw   = additional_datasets.get("validated", Dataset.from_dict({}))
invalidated_raw = additional_datasets.get("invalidated", Dataset.from_dict({}))


In [12]:
from datasets import concatenate_datasets

datasets_to_merge = []
COLUMNS_WE_WANT = ["path", "sentence", "duration"]

for d in [train_raw, dev_raw, validated_raw, invalidated_raw]:
    if len(d) == 0:
        continue
    cols_in_d = [c for c in COLUMNS_WE_WANT if c in d.column_names]
    d = d.select_columns(cols_in_d)
    datasets_to_merge.append(d)

if datasets_to_merge:
    # ensure same columns in all datasets
    shared_cols = set(datasets_to_merge[0].column_names)
    for d_item in datasets_to_merge[1:]:
        shared_cols &= set(d_item.column_names)
    shared_cols = list(shared_cols)

    datasets_to_merge = [d_item.select_columns(shared_cols) for d_item in datasets_to_merge]
    train_full = concatenate_datasets(datasets_to_merge)
else:
    train_full = Dataset.from_dict({})

print("Merged training set size:", len(train_full))
print("Columns:", train_full.column_names)


Merged training set size: 3705
Columns: ['duration', 'sentence', 'path']


In [13]:
import json
import re

# Ibibio digraphs that must be treated as single symbols
DIGRAPHS = ["kp", "gb", "ny", "nw", "kw", "gw", "ng"]

# Normalize text
def normalize_text(s):
    s = s.lower().strip()
    s = s.replace("’", "'")

    # keep ibibio vowels with diacritics
    allowed = "abcdefghijklmnopqrstuvwxyzáéíóúàèìòùọụñʌ '"
    s = ''.join(ch for ch in s if ch in allowed)

    # collapse spaces
    s = re.sub(r"\s+", " ", s)
    return s

# Step 1: normalize
train_text = [normalize_text(t) for t in train_full["sentence"]]

# Step 2: grapheme-tokenize (detect digraphs first)
graphemes = set()

for sentence in train_text:
    i = 0
    while i < len(sentence):
        # skip spaces
        if sentence[i] == " ":
            graphemes.add(" ")
            i += 1
            continue

        # try matching digraph
        matched = False
        for dg in DIGRAPHS:
            if sentence[i:i+len(dg)] == dg:
                graphemes.add(dg)
                i += len(dg)
                matched = True
                break

        if not matched:
            graphemes.add(sentence[i])
            i += 1

# Step 3: sort graphemes, placing space first
graphemes = sorted(list(graphemes))

# Step 4: build vocab
vocab = {g: i for i, g in enumerate(graphemes)}

# Add special CTC tokens
vocab["|"] = len(vocab)       # blank token
vocab["[UNK]"] = len(vocab)
vocab["[PAD]"] = len(vocab)

# Save
with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab, f, ensure_ascii=False, indent=2)

print("FINAL VOCAB:", vocab)
print("SIZE:", len(vocab))


FINAL VOCAB: {' ': 0, 'a': 1, 'b': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'k': 9, 'kp': 10, 'kw': 11, 'm': 12, 'n': 13, 'nw': 14, 'ny': 15, 'o': 16, 'p': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'w': 22, 'y': 23, 'ñ': 24, 'ʌ': 25, 'ọ': 26, '|': 27, '[UNK]': 28, '[PAD]': 29}
SIZE: 30


In [14]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

VOCAB_PATH = "vocab.json"
MODEL_NAME = "facebook/wav2vec2-xls-r-300m"  # or whatever you're using

tokenizer = Wav2Vec2CTCTokenizer(
    VOCAB_PATH,
    unk_token="[UNK]",
    pad_token="[PAD]",
    word_delimiter_token="|",
)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True,
)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
)


from dataclasses import dataclass
from typing import Any, Dict, List
from jiwer import wer
print("TRAIN FULL:", len(train_full))
print("TEST:", len(test_raw))


TRAIN FULL: 3705
TEST: 328


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [15]:
train_full = train_full.shuffle(seed=42)
split = train_full.train_test_split(test_size=0.05, seed=42)
train_ds = split["train"]
eval_ds  = split["test"]

print("Train:", len(train_ds), "Eval:", len(eval_ds), "Test:", len(test_raw))


Train: 3519 Eval: 186 Test: 328


In [16]:
from datasets import Audio

train_ds = train_ds.cast_column("path", Audio(sampling_rate=16000))
eval_ds  = eval_ds.cast_column("path",  Audio(sampling_rate=16000))
test_ds  = test_raw.cast_column("path", Audio(sampling_rate=16000))

def normalize_text_for_labels(s: str) -> str:
    # <-- use the SAME normalize_text you used when building the vocab
    s = s.lower()
    s = s.replace("’", "'")

    # Ibibio digraphs that must be treated as single symbols
    DIGRAPHS = ["kp", "gb", "ny", "nw", "kw", "gw", "ng"] # ensure this matches global DIGRAPHS
    # Reimplement the grapheme logic used in vocab creation
    # Here we are just normalizing, not tokenizing into graphemes
    # The tokenizer will handle the grapheme mapping based on the vocab.json

    allowed = "abcdefghijklmnopqrstuvwxyzáéíóúàèìòùọụñʌ '"
    s = ''.join(ch for ch in s if ch in allowed)

    # collapse spaces
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def prepare_dataset(batch):
    # 1) audio to input_values
    audio = batch["path"]
    batch["input_values"] = processor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_values[0]

    # 2) normalize text & filter
    text = normalize_text_for_labels(batch["sentence"])
    batch["sentence"] = text

    # if text becomes empty, mark labels None (we'll filter later)
    if text == "":
        batch["labels"] = None
        return batch

    # 3) text -> label ids
    # Use the recommended way to process labels without as_target_processor
    batch["labels"] = processor(text=text).input_ids

    return batch

# Apply to each split
train_proc = train_ds.map(
    prepare_dataset,
    remove_columns=[c for c in train_ds.column_names if c not in ["sentence", "input_values", "labels"]],
)

eval_proc = eval_ds.map(
    prepare_dataset,
    remove_columns=[c for c in eval_ds.column_names if c not in ["sentence", "input_values", "labels"]],
)

test_proc = test_ds.map(
    prepare_dataset,
    remove_columns=[c for c in test_ds.column_names if c not in ["sentence", "input_values", "labels"]],
)


Map:   0%|          | 0/3519 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

In [17]:
def keep_valid(example):
    return (
        example["labels"] is not None
        and isinstance(example["labels"], list)
        and len(example["labels"]) > 0
        and all(t is not None for t in example["labels"])
    )

train_proc = train_proc.filter(keep_valid)
eval_proc = eval_proc.filter(keep_valid)
test_proc = test_proc.filter(keep_valid)

print(train_proc[0])


Filter:   0%|          | 0/3519 [00:00<?, ? examples/s]

Filter:   0%|          | 0/186 [00:00<?, ? examples/s]

Filter:   0%|          | 0/328 [00:00<?, ? examples/s]

{'sentence': 'ndion ekpak ufọkmbet ukara sted ebo nọ enam mbet', 'input_values': [9.093301923712716e-05, 9.093310654861853e-05, 9.093269181903452e-05, 9.093288099393249e-05, 9.093271364690736e-05, 9.093283006222919e-05, 9.093284461414441e-05, 9.093358676182106e-05, 9.093287371797487e-05, 9.09324808162637e-05, 9.093290282180533e-05, 9.093454718822613e-05, 9.093418339034542e-05, 9.093522385228425e-05, 9.093834523810074e-05, 9.093146945815533e-05, 9.092810068977997e-05, 9.093156404560432e-05, 9.093520930036902e-05, 9.093518019653857e-05, 9.093918197322637e-05, 9.093682456295937e-05, 9.092700202018023e-05, 9.09355076146312e-05, 9.095019777305424e-05, 9.093168046092615e-05, 9.091881656786427e-05, 9.09387890715152e-05, 9.093999688047916e-05, 9.093342669075355e-05, 9.095593122765422e-05, 9.093296102946624e-05, 9.088806109502912e-05, 9.092812251765281e-05, 9.100356692215428e-05, 9.094161941902712e-05, 9.093947301153094e-05, 9.097978909267113e-05, 9.096727444557473e-05, 9.090403182199225e-05, 9

Data Preprocessing

In [18]:
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import torch

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        # Filter out any sample that somehow has bad labels or inputs
        clean_features = []
        for f in features:
            if (
                f.get("input_values") is not None
                and f.get("labels") is not None
                and isinstance(f["labels"], list)
                and len(f["labels"]) > 0
                and all(t is not None for t in f["labels"])
            ):
                clean_features.append(f)

        if len(clean_features) == 0:
            raise ValueError("All features in batch had invalid labels or inputs.")

        # Split inputs and labels
        input_features = [{"input_values": f["input_values"]} for f in clean_features]
        label_features = [{"input_ids": f["labels"]} for f in clean_features]

        # Pad inputs
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Pad labels
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # Replace padding with -100 to ignore in loss
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        batch["labels"] = labels
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor)


In [26]:
from transformers import TrainingArguments, Trainer, Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    MODEL_NAME,
    vocab_size=len(processor.tokenizer),
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Ibibio_Voice/wav2vec2",
    group_by_length=True,
    per_device_train_batch_size=2,  # Reduced batch size
    per_device_eval_batch_size=2,   # Reduced batch size
    gradient_accumulation_steps=8,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs=18,           # start small; you can increase later
    fp16=torch.cuda.is_available(),
    save_steps=500,
    logging_strategy="steps",
    logging_steps=50,               # VERY IMPORTANT for TensorBoard
    eval_steps=200,

    learning_rate=1e-4,
    save_total_limit=2,

    report_to=["tensorboard"],

    # reduce GPU fragmentation
    dataloader_num_workers=2,
)

model.gradient_checkpointing_enable()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
import numpy as np
from jiwer import wer, cer

def compute_metrics(pred):
    # get logits & labels
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # replace -100 with pad_token_id before decoding
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    # decode
    preds = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels = processor.batch_decode(pred.label_ids, skip_special_tokens=True)

    # compute metrics
    wer_value = wer(labels, preds)
    cer_value = cer(labels, preds)

    return {
        "wer": wer_value,
        "cer": cer_value,
    }


In [28]:
import os
import json
import matplotlib.pyplot as plt
from transformers import TrainerCallback

class SavePlotsCallback(TrainerCallback):
    """
    Saves training graphs (loss, WER, CER) to PNG files
    inside the training output directory.
    """

    def on_train_end(self, args, state, control, **kwargs):
        output_dir = args.output_dir
        os.makedirs(output_dir, exist_ok=True)

        # --------------------------
        # Load Trainer's metrics log
        # --------------------------
        log_history = state.log_history

        train_loss = []
        eval_loss = []
        wer_vals = []
        cer_vals = []
        steps = []

        for entry in log_history:
            if "loss" in entry and "epoch" in entry:
                train_loss.append(entry["loss"])
                steps.append(entry["step"])
            if "eval_loss" in entry:
                eval_loss.append(entry["eval_loss"])
            if "eval_wer" in entry:
                wer_vals.append(entry["eval_wer"])
            if "eval_cer" in entry:
                cer_vals.append(entry["eval_cer"])

        # --------------------------
        # Save Training Loss Plot
        # --------------------------
        if train_loss:
            plt.figure()
            plt.plot(train_loss)
            plt.title("Training Loss Curve")
            plt.xlabel("Logging Step")
            plt.ylabel("Loss")
            plt.grid(True)
            plt.savefig(os.path.join(output_dir, "training_loss.png"))
            plt.close()

        # --------------------------
        # Save Eval Loss Plot
        # --------------------------
        if eval_loss:
            plt.figure()
            plt.plot(eval_loss)
            plt.title("Validation Loss Curve")
            plt.xlabel("Evaluation Step")
            plt.ylabel("Loss")
            plt.grid(True)
            plt.savefig(os.path.join(output_dir, "validation_loss.png"))
            plt.close()

        # --------------------------
        # Save WER Plot
        # --------------------------
        if wer_vals:
            plt.figure()
            plt.plot(wer_vals)
            plt.title("Word Error Rate (WER)")
            plt.xlabel("Evaluation Step")
            plt.ylabel("WER")
            plt.grid(True)
            plt.savefig(os.path.join(output_dir, "wer.png"))
            plt.close()

        # --------------------------
        # Save CER Plot
        # --------------------------
        if cer_vals:
            plt.figure()
            plt.plot(cer_vals)
            plt.title("Character Error Rate (CER)")
            plt.xlabel("Evaluation Step")
            plt.ylabel("CER")
            plt.grid(True)
            plt.savefig(os.path.join(output_dir, "cer.png"))
            plt.close()

        print(f"\n📊 Saved training plots into: {output_dir}\n")


In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_proc,
    eval_dataset=eval_proc,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
    callbacks=[SavePlotsCallback()],
)

/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [30]:
import torch
torch.cuda.empty_cache()
print("GPU memory cleared.")

GPU memory cleared.


In [31]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reent

Epoch,Training Loss,Validation Loss,Wer,Cer
1,2.877800,2.885374,1.000000,1.000000
2,2.904700,2.779813,1.000000,1.000000
3,1.091900,0.715756,0.674863,0.167510
4,0.641700,0.415364,0.418423,0.100813
5,0.496300,0.329459,0.343872,0.078274
6,0.532000,0.386186,0.298205,0.067004
7,0.672500,0.441784,0.322795,0.069611
8,0.618000,0.415497,0.344262,0.077430
9,0.562100,0.374336,0.317330,0.072907
10,0.515700,0.361958,0.311475,0.073137


/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call


📊 Saved training plots into: /content/drive/MyDrive/Ibibio_Voice/wav2vec2



TrainOutput(global_step=3960, training_loss=0.9452840842381872, metrics={'train_runtime': 5207.0702, 'train_samples_per_second': 12.165, 'train_steps_per_second': 0.761, 'total_flos': 1.7552089588031631e+19, 'train_loss': 0.9452840842381872, 'epoch': 18.0})

In [25]:
model.save("/content/drive/MyDrive/Ibibio_Voice/wav2vec2")

AttributeError: 'Wav2Vec2ForCTC' object has no attribute 'save'

In [ ]:
import librosa
import numpy as np

SAMPLE_RATE = 16000

def transcribe_example(example):
    # 1. Load audio
    path = example["path"]
    speech, sr = librosa.load(path, sr=SAMPLE_RATE)

    # 2. Processor → model
    inputs = processor(
        speech,
        sampling_rate=SAMPLE_RATE,
        return_tensors="pt",
        padding=True
    )

    with torch.no_grad():
        logits = model(inputs.input_values.to(device)).logits

    pred_ids = torch.argmax(logits, dim=-1)
    pred_str = processor.batch_decode(pred_ids)[0]

    # 3. Reference
    gold_str = example["sentence"]

    return gold_str, pred_str


In [ ]:
from random import sample

# choose N random indices or just first N
indices = sample(range(len(test_raw)), 5)

for idx in indices:
    example = test_raw[idx]

    # extract file path — handles multiple path formats
    if isinstance(example["path"], dict) and "path" in example["path"]:
        file_path = example["path"]["path"]
    elif hasattr(example["path"], "path"):
        file_path = example["path"].path
    else:
        file_path = example["path"]

    gold, pred = transcribe_example(example)

    print(f"Example {idx}")
    print(f"  FILE: {file_path}")
    print(f"  GOLD: {gold}")
    print(f"  PRED: {pred}")
    print("_" * 60)


In [ ]:
bad = [s for s in train_full["sentence"] if s is None or s.strip() == ""]
len(bad), bad[:10]
